In [6]:
import pandas as pd 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
from sklearn.linear_model import LogisticRegression
import scipy.stats as st
import sklearn.metrics as metrics
from sklearn.model_selection import train_test_split,KFold,StratifiedKFold,GridSearchCV,RandomizedSearchCV,cross_val_score
from sklearn.ensemble import RandomForestClassifier,BaggingClassifier,AdaBoostClassifier,GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import r2_score,roc_auc_score,recall_score,classification_report,mean_squared_error,accuracy_score,confusion_matrix

In [2]:
#loading the data
data_train=pd.read_csv("cleaned_data.csv")
data_test=pd.read_csv("cleaned_data_test.csv")


In [3]:
train,val=train_test_split(data_train,test_size=.20,random_state=42)
train.shape,val.shape

Xfull=train.drop(["INCIDENT_ID","MULTIPLE_OFFENSE","DATE"],axis=1)
yfull=train["MULTIPLE_OFFENSE"]
Xtest=data_test.drop(["INCIDENT_ID","DATE"],axis=1)


Xtrain=train.drop(["INCIDENT_ID","MULTIPLE_OFFENSE","DATE"],axis=1)
ytrain=train["MULTIPLE_OFFENSE"]
print(Xtrain.shape,ytrain.shape)

Xval=val.drop(["MULTIPLE_OFFENSE","INCIDENT_ID","DATE"],axis=1)
yval=val["MULTIPLE_OFFENSE"]
print(Xval.shape,yval.shape)

(19083, 15) (19083,)
(4771, 15) (4771,)


In [4]:

lrc = LogisticRegression()
rfc=RandomForestClassifier()
adbc=AdaBoostClassifier()
bgc=BaggingClassifier()
gnb = GaussianNB()
knn=KNeighborsClassifier()
dtc = DecisionTreeClassifier()
ab_rfc = AdaBoostClassifier(base_estimator=rfc,random_state=42)
ab_dtc = AdaBoostClassifier(base_estimator=dtc,random_state=42)
ab_nbc=  AdaBoostClassifier(base_estimator=gnb,random_state=42)
ab_lrc=  AdaBoostClassifier(base_estimator=lrc,random_state=42)
bgcl_lrc = BaggingClassifier(base_estimator=lrc, random_state=42)


In [9]:
models=[lrc,rfc,adbc,bgc,gnb,knn,dtc,bgcl_lrc]
sctr,scte,auc,ps,rs=[],[],[],[],[]
def ensemble_for_train_val(X_train,X_test, y_train, y_test):
    for model in models:
            model.fit(X_train, y_train)
            y_test_pred = model.predict(X_test)
            y_test_pred_new=model.predict_proba(X_test)
            y_test_pred_new=y_test_pred_new[:,1]
            train_score=model.score(X_train,y_train)
            test_score=model.score(X_test,y_test)
            p_score=metrics.precision_score(y_test,y_test_pred)
            r_score=metrics.recall_score(y_test,y_test_pred)
            
            ac=metrics.roc_auc_score(y_test,y_test_pred_new)
            
            sctr.append(train_score)
            scte.append(test_score)
            ps.append(p_score)
            rs.append(r_score)
            auc.append(ac)
    return sctr,scte,auc,ps,rs

ensemble_for_train_val(Xtrain,Xval, ytrain, yval)
# 'ab_rf','ab_dt','ab_nb','ab_lr','bgcl_lr'
ensemble=pd.DataFrame({'names':['Logistic Regression','Random Forest','Ada boost','Bagging',
                                'Naive-Bayes','KNN','Decistion Tree',
                                'bagged LR'],
                       'auc_score':auc,'training':sctr,'testing':scte,'precision':ps,'recall':rs})
ensemble=ensemble.sort_values(by='recall',ascending=False).reset_index(drop=True)
ensemble

,names,auc_score,training,testing,precision,recall
0,Random Forest,0.999364,1.000000,0.993502,0.994345,0.998908
1,Decistion Tree,0.999454,1.000000,0.998952,1.000000,0.998908
2,Ada boost,0.999700,0.996175,0.996437,0.997817,0.998471
3,Bagging,0.999608,0.999790,0.997275,0.999781,0.997378
4,KNN,0.852580,0.974952,0.971075,0.974353,0.996067
5,Logistic Regression,0.875039,0.954200,0.961224,0.965254,0.995412
6,bagged LR,0.869761,0.954619,0.959757,0.964808,0.994319
7,Naive-Bayes,0.887816,0.934339,0.937120,0.971347,0.962858


In [38]:
models=[lrc,rfc,adbc,bgc,gnb,knn,dtc,bgcl_lrc]
# predlrc=[],pred=dtc[],predrfc=[],predadbc=[],predbgc=[],predgnb=[],predknn=[],preddtc=[],predbgcl_lrc=[]
predictions=pd.DataFrame(data_test['INCIDENT_ID'])
def pred_on_full_data(Xtrain,ytrain,Xtest):
    for model in models:
        print(model.__class__.__name__)
        model.fit(Xtrain, ytrain)
        y_test_pred = model.predict(Xtest)
        predictions=pd.concat([data_test['INCIDENT_ID'],pd.DataFrame(y_test_pred,columns=['MULTIPLE_OFFENSE'])],1)
        a="submission_base"+model.__class__.__name__+".csv"
        predictions.to_csv(a,index=False)

In [39]:
pred_on_full_data(Xfull,yfull,Xtest)

LogisticRegression
RandomForestClassifier
AdaBoostClassifier
BaggingClassifier
GaussianNB
KNeighborsClassifier
DecisionTreeClassifier
BaggingClassifier
